In [ ]:
!pip install datasets
!pip install -U transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DistilBertConfig,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset



# Distillation using Masked Language Modelling

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    AutoModelForSequenceClassification,
)
import torch.optim as opt
from datasets import load_dataset
from tqdm import tqdm
import torch.nn.functional as F


# Finetune for MRPC


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/CS4782FinalProject/artifacts/imdb/distilled-bert-small-wikitext"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5

In [ ]:
mrpc = load_dataset("glue", "mrpc")

model_path="/content/drive/MyDrive/CS4782FinalProject/artifacts/imdb/distilled-bert-small-wikitext"
tokenizer = AutoTokenizer.from_pretrained(model_path,local_files_only=True)

def preprocess(example):
    return tokenizer(
        example["sentence1"],
        example["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=256
    )


encoded_mrpc = mrpc.map(preprocess, batched=True)
encoded_mrpc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)
train_loader = DataLoader(encoded_mrpc["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator)
test_loader = DataLoader(encoded_mrpc["test"], batch_size=BATCH_SIZE, collate_fn=data_collator)


**Load the distilled student model and add a classification head to it.**

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=2).to(DEVICE)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_mrpc["train"],
    eval_dataset=encoded_mrpc["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
save_path = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/finetuned-distilled-mrpc"
model.save_pretrained(save_path)

In [ ]:
tokenizer.save_pretrained(save_path)

# Finetune teacher on MRPC

In [ ]:

# Load teacher model with classification head
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
).to(DEVICE)

teacher_training_args = TrainingArguments(
    output_dir="./results-teacher",
    eval_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs-teacher",
    logging_steps=100,
    report_to="none"
)

teacher_trainer = Trainer(
    model=teacher_model,
    args=teacher_training_args,
    train_dataset=encoded_mrpc["train"],
    eval_dataset=encoded_mrpc["test"],
    tokenizer=tokenizer,  # Use student tokenizer since it was already applied to data
    data_collator=data_collator,
)

In [ ]:
# === Train the teacher ===
teacher_trainer.train()

# === Evaluate the teacher ===
teacher_trainer.evaluate()

In [ ]:
teacher_finetuned_path = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/finetuned-bert-mrpc"
teacher_model.save_pretrained(teacher_finetuned_path)
tokenizer.save_pretrained(teacher_finetuned_path)

# Comparison of both finetuned models

In [ ]:
student_model_path = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/finetuned-distilled-mrpc"
teacher_model_path = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/finetuned-bert-mrpc"


In [ ]:
from transformers import AutoModelForSequenceClassification
student = AutoModelForSequenceClassification.from_pretrained(student_model_path).to(DEVICE)
teacher = AutoModelForSequenceClassification.from_pretrained(teacher_model_path).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(student_model_path)

In [ ]:
encoded_mrpc = mrpc.map(preprocess, batched=True)
encoded_mrpc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

test_dataset = encoded_mrpc["test"]

In [ ]:
def custom_collate(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.tensor([item['label'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

test_loader = DataLoader(encoded_mrpc["test"], batch_size=BATCH_SIZE, collate_fn=custom_collate)


In [ ]:
from tqdm import tqdm
# data_collator = DataCollatorWithPadding(tokenizer)
# test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total

# Evaluate
student_accuracy = evaluate(student, test_loader)
teacher_accuracy = evaluate(teacher, test_loader)

print(f"Student accuracy: {student_accuracy:.4f}\n")
print(f"Teacher accuracy: {teacher_accuracy:.4f}")


In [ ]:
student_size = sum(p.numel() for p in student.parameters()) / 1e6
teacher_size = sum(p.numel() for p in teacher.parameters()) / 1e6

print(f"Student model size: {student_size:.2f}M parameters")
print(f"Teacher model size: {teacher_size:.2f}M parameters")

In [ ]:
import os
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def evaluate_model(model, dataloader, model_path=None):
    model.eval()
    preds = []
    labels = []
    start_time = time.time()

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            batch_labels = batch["labels"].to(DEVICE)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds.append(logits.argmax(dim=-1).cpu())
            labels.append(batch_labels.cpu())

    end_time = time.time()
    total_time = end_time - start_time

    preds = torch.cat(preds)
    labels = torch.cat(labels)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    conf_matrix = confusion_matrix(labels, preds)

    param_count = sum(p.numel() for p in model.parameters())

    # if model_path is not None:
    #     model_size = os.path.getsize(os.path.join(model_path, "pytorch_model.bin")) / (1024**2)  # in MB

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": conf_matrix,
        "inference_time_sec": total_time,
        "param_count": param_count,
    }


In [ ]:
student_results = evaluate_model(student, test_loader, model_path=student_model_path)
teacher_results = evaluate_model(teacher, test_loader, model_path=teacher_model_path)


In [ ]:
import pandas as pd

comparison_df = pd.DataFrame([
    {
        "Model": "Student",
        **student_results,
    },
    {
        "Model": "Teacher",
        **teacher_results,
    }
])


comparison_df = comparison_df[[
    "Model",
    "accuracy",
    "precision",
    "recall",
    "f1",
    "inference_time_sec",
    "param_count"
]]


print(comparison_df.to_markdown(index=False))


In [ ]:
save_path = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/plots/comparison_results.csv"
comparison_df.to_csv(save_path, index=False)
print(f"Saved CSV to {save_path}")

In [ ]:
import matplotlib.pyplot as plt
import os


plots_dir = "/content/drive/MyDrive/CS4782FinalProject/artifacts/mrpc/plots"
os.makedirs(plots_dir, exist_ok=True)


metrics = ["accuracy", "f1", "inference_time_sec", "param_count"]

for metric in metrics:
    plt.figure(figsize=(6, 4))
    plt.bar(comparison_df["Model"], comparison_df[metric])
    plt.title(f"Comparison: {metric}")
    plt.ylabel(metric)
    plt.show()

    save_path = os.path.join(plots_dir, f"{metric}_comparison.png")
    plt.savefig(save_path)

    plt.close()

In [ ]:
print(student_results)
print(teacher_results)